In [1]:
import csv
import pandas as pd
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
from matplotlib import font_manager
from collections import Counter
%matplotlib inline

### 读取数据并查看

In [3]:
raw_data = pd.read_csv('raw_data.csv')

In [4]:
data = raw_data.copy()

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
title            100 non-null object
actor            100 non-null object
releasetime      100 non-null object
stars            100 non-null float64
style            100 non-null object
location         100 non-null object
film_duration    100 non-null object
dtypes: float64(1), object(6)
memory usage: 5.5+ KB


In [6]:
data.head(10)

,title,actor,releasetime,stars,style,location,film_duration
0,霸王别姬,"张国荣,张丰毅,巩俐",1993-01-01,9.5,"爱情,剧情","中国大陆,中国香港",171分钟
1,肖申克的救赎,"蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿",1994-09-10,9.5,"犯罪,剧情",美国,142分钟
2,罗马假日,"格利高里·派克,奥黛丽·赫本,埃迪·艾伯特",1953-09-02,9.1,"喜剧,剧情,爱情",美国,118分钟
3,这个杀手不太冷,"让·雷诺,加里·奥德曼,娜塔莉·波特曼",1994-09-14,9.5,"剧情,惊悚,犯罪",法国,110分钟
4,泰坦尼克号,"莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩",1998-04-03,9.5,"历史,爱情,灾难",美国,194分钟
5,唐伯虎点秋香,"周星驰,巩俐,郑佩佩",1993-07-01,9.1,"喜剧,爱情,古装",中国香港,102分钟
6,魂断蓝桥,"费雯·丽,罗伯特·泰勒,露塞尔·沃特森",1940-05-17,9.2,"剧情,战争,爱情",美国,108分钟
7,乱世佳人,"费雯·丽,克拉克·盖博,奥利维娅·德哈维兰",1939-12-15,9.1,"剧情,爱情,战争",美国,238分钟
8,天空之城,"寺田农,鹫尾真知子,龟山助清",1992,9.1,"动画,冒险,家庭",日本,125分钟
9,辛德勒的名单,"连姆·尼森,拉尔夫·费因斯,本·金斯利",1993-12-15,9.2,"剧情,历史,战争",美国,195分钟


### 清洗数据

In [7]:
# 发行日期栏 更改数据类型为 datetime
data['releasetime'] = pd.to_datetime(data['releasetime'])

In [8]:
# 增加两栏来统计 发型日期的年份 和月份
data['release-year'] = data['releasetime'].dt.year
data['release-month'] = data['releasetime'].dt.month

In [9]:
# 电影时长栏 数据清洗变为 以分钟计数的数字
data.rename(columns={'film_duration':'film_duration(minutes)'},inplace=True)
data['film_duration(minutes)'] = data['film_duration(minutes)'].str.strip('分钟')
data['film_duration(minutes)'] = data['film_duration(minutes)'].astype(int)

In [10]:
# 清洗电影拍摄地一栏数据
data['location'] = data['location'].str.split(',',expand=True)[0]

In [11]:
data['location'].value_counts()

美国      46
中国香港    13
日本       9
法国       6
韩国       6
意大利      5
英国       4
中国大陆     4
德国       1
印度       1
中国台湾     1
泰国       1
伊朗       1
新西兰      1
巴西       1
Name: location, dtype: int64

In [12]:
data.loc[data['location'] == '中国香港', 'location'] = '中国'
data.loc[data['location'] == '中国大陆', 'location'] = '中国'
data.loc[data['location'] == '中国台湾', 'location'] = '中国'

In [13]:
data['location'].value_counts()

美国     46
中国     18
日本      9
法国      6
韩国      6
意大利     5
英国      4
德国      1
印度      1
伊朗      1
新西兰     1
泰国      1
巴西      1
Name: location, dtype: int64

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
title                     100 non-null object
actor                     100 non-null object
releasetime               100 non-null datetime64[ns]
stars                     100 non-null float64
style                     100 non-null object
location                  100 non-null object
film_duration(minutes)    100 non-null int64
release-year              100 non-null int64
release-month             100 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 7.1+ KB


In [15]:
data.head(10)

,title,actor,releasetime,stars,style,location,film_duration(minutes),release-year,release-month
0,霸王别姬,"张国荣,张丰毅,巩俐",1993-01-01,9.5,"爱情,剧情",中国,171,1993,1
1,肖申克的救赎,"蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿",1994-09-10,9.5,"犯罪,剧情",美国,142,1994,9
2,罗马假日,"格利高里·派克,奥黛丽·赫本,埃迪·艾伯特",1953-09-02,9.1,"喜剧,剧情,爱情",美国,118,1953,9
3,这个杀手不太冷,"让·雷诺,加里·奥德曼,娜塔莉·波特曼",1994-09-14,9.5,"剧情,惊悚,犯罪",法国,110,1994,9
4,泰坦尼克号,"莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩",1998-04-03,9.5,"历史,爱情,灾难",美国,194,1998,4
5,唐伯虎点秋香,"周星驰,巩俐,郑佩佩",1993-07-01,9.1,"喜剧,爱情,古装",中国,102,1993,7
6,魂断蓝桥,"费雯·丽,罗伯特·泰勒,露塞尔·沃特森",1940-05-17,9.2,"剧情,战争,爱情",美国,108,1940,5
7,乱世佳人,"费雯·丽,克拉克·盖博,奥利维娅·德哈维兰",1939-12-15,9.1,"剧情,爱情,战争",美国,238,1939,12
8,天空之城,"寺田农,鹫尾真知子,龟山助清",1992-01-01,9.1,"动画,冒险,家庭",日本,125,1992,1
9,辛德勒的名单,"连姆·尼森,拉尔夫·费因斯,本·金斯利",1993-12-15,9.2,"剧情,历史,战争",美国,195,1993,12


In [16]:
data.to_csv('data.csv',index=None)